In [1]:
%load_ext autoreload
%autoreload 2

In [40]:
import scipy
import numpy as np
import torch
import matplotlib.pyplot as plt
import pdb
from tqdm import tqdm
from neurosim.models.varma import gen_var1_connectivity
from neurosim.models.ssr import StateSpaceRealization as SSR
from neurosim.models.ssr import memoized
import pandas as pd

In [23]:
from dca.dca import DynamicalComponentsAnalysis as DCA
from dca_research.kca import KalmanComponentsAnalysis as KCA
from sklearn.decomposition import PCA
import collections
from dca.dca import init_coef, ObjectiveWrapper, ortho_reg_fn
from scipy.optimize import minimize
from dca.cov_util import calc_pi_from_data

In [4]:
from pseudopy.nonnormal import NonnormalAuto, NonnormalPoints

In [5]:
from neurosim.models.subspaces import residue_calc
from neurosim.models.subspaces import measure_angle

In [6]:
import pickle
import h5py

In [7]:
f = h5py.File('/media/akumar/Data/nse/data/neuraldata/EC2_hg.h5', 'r')

In [8]:
f['X']

<HDF5 dataset "X": shape (2576, 258, 86, 1), type "<f4">

In [9]:
f['y'][:]

array([b'raa', b'shee', b'yoo', ..., b'kaa', b'waa', b'daa'], dtype='|S4')

In [ ]:
# Count how many of each category there are

In [10]:
# Use logistic regression to classify f['y'] (the particular syllable) using f['X']
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

In [ ]:
# No normalization for now
# X = np.squeeze(f['X'])
# scaler = StandardScaler()
# Xstd = np.zeros(X.shape)
# for i in range(X.shape[0]):
#     for j in range(X.shape[2])
#     Xstd[i, :, j] = scaler.fit_transform(X[i, :, j])

In [11]:
classifier = LogisticRegression()

In [12]:
X = np.squeeze(f['X'])
y = np.array(f['y'])

In [16]:
syllable_counts = {}
n_folds = 10
cv = StratifiedKFold(n_folds, shuffle=True, random_state=1234)

# Initialize keys
for elem in np.unique(y):
    syllable_counts[elem] = np.zeros(n_folds)

fold_idx = 0
for train_idxs, test_idxs in cv.split(X, y):
    yy = y[train_idxs]
    
    for elem in yy:
        syllable_counts[elem][fold_idx] += 1
        
    fold_idx += 1

In [17]:
np.where(y != b'thee')[0].size

2566

In [18]:
# Remove 'thee'
theeless = np.where(y != b'thee')[0]
X = X[theeless, ...]
y = y[theeless]

In [19]:
cv = StratifiedKFold(10, shuffle=True, random_state=1234)
scores = []
for train_idxs, test_idxs in cv.split(X, y):
    xx = np.reshape(X[train_idxs, ...], (train_idxs.size, -1))
    
#     scaler = StandardScaler()
#     xx = scaler.fit_transform(xx)
    
    classifier.fit(xx, yy)
    scores.append(classifier.score(np.reshape(X[test_idxs, ...], (test_idxs.size, -1)), y[test_idxs]))

/home/akumar/anaconda3/envs/dyn/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/akumar/anaconda3/envs/dyn/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modu

In [20]:
np.mean(scores)

0.41387858706225683

In [21]:
scores

[0.4163424124513619,
 0.38910505836575876,
 0.4591439688715953,
 0.4085603112840467,
 0.4046692607003891,
 0.39299610894941633,
 0.43359375,
 0.4140625,
 0.42578125,
 0.39453125]

In [24]:
X.shape

(2566, 258, 86)

In [60]:
# Do dimreduc and decoding for CV dataset right here
dimreduc_results = []
dim_vals = np.arange(1, 25) + 1
T = [1, 2, 3]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for i in tqdm(range(dim_vals.size)):
    for T_ in T:
        fold_idx = 0
        for train_idxs, test_idxs in cv.split(X, y):
            xtrain = X[train_idxs]
 
            dcamodel = DCA(d=dim_vals[i], T=T_)
            kcamodel = KCA(d=dim_vals[i], T=T_, causal_weights=(1, 0), project_mmse=False)
            pcamodel = PCA()

            dcamodel.fit(xtrain)
            kcamodel.fit(xtrain)
            pcamodel.fit(np.reshape(xtrain, (-1, xtrain.shape[-1])))

            results = {}
            results['T'] = T_
            results['fold_idx'] = fold_idx
            results['dim'] = dim_vals[i]

            results['DCA'] = {}
            results['KCA'] = {}
            results['PCA'] = {}

            results['DCA']['coef'] = dcamodel.coef_
            results['DCA']['score'] = dcamodel.score(xtrain)

            results['KCA']['coef'] = kcamodel.coef_
            results['KCA']['score'] = kcamodel.score() 

            if dim_vals[i] > 1:
                results['PCA']['coef'] = pcamodel.components_.T[:, 0:dim_vals[i]]
            else:
                results['PCA']['coef'] = pcamodel.components_.T[:, 0][:, np.newaxis]

            results['PCA']['score'] = np.sum(pcamodel.explained_variance_ratio_[0:dim_vals[i]])

            dimreduc_results.append(results)

            fold_idx += 1

  0%|          | 0/24 [00:00<?, ?it/s]/home/akumar/nse/DCA_research/dca_research/kca.py:311: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cross_cov_mats = torch.tensor(cross_cov_mats)
/home/akumar/nse/DCA_research/dca_research/kca.py:311: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cross_cov_mats = torch.tensor(cross_cov_mats)
/home/akumar/nse/DCA_research/dca_research/kca.py:311: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cross_cov_mats = torch.tensor(cross_cov_mats)
/home/akumar/nse/DCA_research/dca_research/kca.py:311: UserWarnin

In [61]:
dimreduc_results = pd.DataFrame(dimreduc_results)

In [62]:
with open('cv_dimreduc_results.dat', 'wb') as f:
    f.write(pickle.dumps(dimreduc_results))

In [63]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
split = list(cv.split(X, y))

In [64]:
decoding_results = []

for i in tqdm(range(dimreduc_results.shape[0])):

    for dimreduc in ['PCA', 'DCA', 'KCA']:
        results = {}
        
        results['T'] = dimreduc_results.iloc[i]['T']
        results['fold_idx'] = dimreduc_results.iloc[i]['fold_idx']
        results['dim'] = dimreduc_results.iloc[i]['dim']

        results['dimreduc_method'] = dimreduc

        xtrain = X[split[dimreduc_results.iloc[i]['fold_idx']][0]]
        ytrain = y[split[dimreduc_results.iloc[i]['fold_idx']][0]]

        xtest = X[split[dimreduc_results.iloc[i]['fold_idx']][1]]
        ytest = y[split[dimreduc_results.iloc[i]['fold_idx']][1]]
        
        xtrain = xtrain @ dimreduc_results.iloc[i][dimreduc]['coef']
        xtest = xtest @ dimreduc_results.iloc[i][dimreduc]['coef']

        classifier = LogisticRegression()
        classifier.fit(np.reshape(xtrain, (xtrain.shape[0], -1)), ytrain)       
        
        score = classifier.score(np.reshape(xtest, (xtest.shape[0], -1)), ytest)

        results['score'] = score
        decoding_results.append(results)

  0%|          | 0/360 [00:00<?, ?it/s]/home/akumar/anaconda3/envs/dyn/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/akumar/anaconda3/envs/dyn/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

In [65]:
decoding_results = pd.DataFrame(decoding_results)

In [66]:
with open('cv_decoding_results.dat', 'wb') as f:
    f.write(pickle.dumps(decoding_results))